# Minitarea 2

Nombre: Javier Urrutia

---------------------------
## Language Models (3 pts)
Estas preguntas son teóricas y deben ser resueltas con desarrollo, pero sin código. Por favor usen $\LaTeX$ para las fórmulas. El material del curso y los videos deberian ser suficientes para que puedan responder.


### Pregunta 1 (1 pt)
Asuma que tiene un modelo de lenguaje de trigramas (simple) entrenado sobre un corpus C. Muestre cómo el modelo le asigna una probabilidad a la oración `el perro le ladra al gato` usando los parámetros estimados a partir del corpus  $q(w_i | w_{i-2}, w_{i-1})$ .

**Respuesta:** La probabilidad de la oracion completa será $q("el"|"*","*")*q("perro"|"*","el")*q("le"|"el","perro")*q("ladra"|"perro","le")*q("al"|"le","ladra")*q("gato"|"ladra","al")$

### Pregunta 2 (1 pt)
Muestre cómo se calcularía  $q(w_{i} | w_{i-2}, w_{i-1})$  usando un modelo que interpola un modelo de lenguajes de trigramas, bigramas, y unigrama.

**Respuesta:** En un modelo que interpole se tiene que: $q(w_{i} | w_{i-2}, w_{i-1})=\lambda_1*q_{tri}(w_{i} | w_{i-2}, w_{i-1}) + \lambda_2*q_{bi}(w_{i} | w_{i-1}) + \lambda_3*q_{uni}(w_{i} )$ donde $\lambda_1+\lambda_2+\lambda_3=1$ y $\lambda_1,\lambda_2,\lambda_3 \geq 0$

### Pregunta 3 (1 pt)
¿Qué ventajas tiene el modelo interpolado sobre el modelo de trigramas simple?

**Respuesta:** Una ventaja es que van a existir varios trigramas o bigramas que no se verán en el corpus de entrenamiento, por lo que al interpolar se evitan las indefiniciones. Además el modelo interpolado logra un mejor balance entre sesgo y varianza en comparacion al trigrama simple que es un modelo con mas variabilidad. 

-----------------------
## Naive Bayes (3 pts)
En esta parte de la minitarea deberan programar Naive Bayes Multinomial usando Laplace Smothing. Las referencias las pueden encontrar en el material del curso y los videos del profesor.

A continuacion se presentan un conjunto de documentos de training divididos en 2 categorias distintas. Ustedes deberan clasificar los documentos del test set usando Naive Bayes con Laplace Smothing.

Por favor, documenten su codigo. Escriban que hacen las funciones, que reciben, que entregan, etc. Si en algun lugar de su codigo hacen algo que creen que debe ser explicado, pongan comentarios, son bienvenidos. 


**Underflow prevention:** En vez de hacer muchas multiplicacions de `float`s, reemplacenlas por sumas de logaritmos para prevenir errores de precision. Revisen la diapo 69 de las slides. 

NOTA: Sobre las `namedtuple`s. Es el tipo de los documentos. Son objetos inmutable que tienen dos atributos: `words` donde estan las palabras del documento y `class_` donde se guarda la clase de ese documento. Estos objetos son inmutables, lo que quiere decir que si quieren modificar un documento y cambiarle la clase, tienen que crear otro documento. Otra cosa es que son tuplas como cualquier otra, es decir se pueden acceder usando indices como `doc[0]` o `doc[1]`. Son libres de implementar su solucion como quieran, si no les gusta este tipo de dato usen otro.



In [8]:
from pprint import pprint
from collections import namedtuple
document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

train_set = (
    document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
    document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
    document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
    document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
    document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
    document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
    document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
    document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1)
)
print("Train documents:")
pprint(train_set)


test_set = (document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)
print("\nTest documents:")
pprint(test_set)

Train documents:
(document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
 document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
 document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
 document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
 document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
 document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
 document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
 document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1))

Test documents:
(document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)


In [56]:
import numpy as np
# Tu respuesta
def get_pc(train_set):
    """
    Get p(c) for each class in the training set
    input:
        train_set: training corpus of document type
    """
    countC = {}
    # count number of documents for each class
    for d in train_set:
        c = d.class_
        if c not in countC:
            countC[c] = 1
        else:
            countC[c] +=1
    Pc = {}
    # Calculate prior for each class
    for class_ in countC:
        Pc[class_] = countC[class_]/len(train_set)
    return Pc

def get_vocab(train_set):
    """
    Get the vocabulary in a training set corpus
    input:
        train_set: training corpus of document type
    """
    vocab = []
    for d in train_set:
        for w in d.words:
            if w not in vocab:
                vocab.append(w)
    return vocab

def get_word_p(train_set, classes):
    """
    Get P(w|c) for each word and class with Laplace smoothing
    input:
        train_set: training corpus of document type
        classes: Classes in the training set as python iterable
    """
    # get corpus vocabulary
    vocab = get_vocab(train_set)
    # Store P(w|c) as dictionary for each class
    Pw = {}
    for c in classes:
        Pw[c] = {}
        # Make single document from elements of the same class
        doc = []
        for d in train_set:
            if d.class_ == c:
                doc.extend(d.words)
        # Calculate probability for each word
        for word in vocab:
            Pw[c][word] = (doc.count(word)+1)/(len(doc)+len(vocab))
        # Probability for word outside of vocabulary
        Pw[c]["_?_"] = 1/(len(doc)+len(vocab))
    return Pw

def multinomial_NB(doc ,Pc, Pw):
    """
    Classify text using multinomial Naive Bayes
    input:
        doc: A document to classify
        Pc: p(c) for each class from training set in dictionary form
        Pw: P(w|c) for each word and class with Laplace smoothing in dictionary form
    """
    Pd = {}
    for class_ in Pc:
        # Start with prior
        log_prob = np.log2(Pc[class_])
        for word in doc.words:
            # Multiply by probability of each word, or the default if the word is new, given the class
            log_prob += np.log2(Pw[class_].get(word, Pw[class_]["_?_"]))
        Pd[class_] = log_prob
    # Find most likely class
    MLclass = max(Pd, key=Pd.get)
    return MLclass

def classify_set_NB(test_set, Pc, Pw):
    """
    Classify text from a set with multinomial Naive Bayes
    input:
        test_set: corpus of document type
        Pc: p(c) for each class from training set in dictionary form
        Pw: P(w|c) for each word and class with Laplace smoothing in dictionary form
    """
    result = [multinomial_NB(doc, Pc, Pw) for doc in test_set]
    return result

In [57]:
# Train NB model
Pc = get_pc(train_set)
Pw = get_word_p(train_set, classes=Pc.keys())

In [58]:
# Classify example
doc_class = classify_set_NB(test_set, Pc, Pw)
print("Predicted classes")
print(doc_class)

Predicted classes
[0]
